In [1]:
!pip install gymnasium

In [2]:
!pip install --upgrade keras-rl2

In [3]:
!pip install tensorflow==2.10 # keras-rl2 is compatible with tensorflow 2.10

In [4]:
import gym
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input, Dense, Activation, Reshape
from tensorflow.keras.optimizers import Adam
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.agents.dqn import DQNAgent # This import should work now

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/usr/local/lib/python3.10/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [5]:
env = gym.make('CartPole-v1')
env.max_episode_steps = 500
num_actions = env.action_space.n

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [6]:
from keras.callbacks import TensorBoard

class CustomTensorBoard(TensorBoard):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._should_trace = False  # Add the missing attribute

# Initialize the custom TensorBoard callback
tb = CustomTensorBoard(log_dir='./logs')  # Adjust the log directory as needed

In [7]:
# Creating out simple NN model here
observation = Input(shape=(1, ) + env.observation_space.shape)
x = Dense(16, activation='relu')(observation)
x = Dense(16, activation='relu')(x)
x = Dense(16, activation='relu')(x)
output = Dense(num_actions, activation='linear')(x)
output = Reshape((num_actions, ))(output)
model = Model(inputs=observation, outputs=output)
print(model.summary())

/usr/lib/python3.10/random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 4)]            0         
                                                                 
 dense (Dense)               (None, 1, 16)             80        
                                                                 
 dense_1 (Dense)             (None, 1, 16)             272       
                                                                 
 dense_2 (Dense)             (None, 1, 16)             272       
                                                                 
 dense_3 (Dense)             (None, 1, 2)              34        
                                                                 
 reshape (Reshape)           (None, 2)                 0         
                                                                 
Total params: 658
Trainable params: 658
Non-trainable params:

In [8]:
memory = SequentialMemory(limit=100000, window_length=1)

In [9]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1, value_min=0.1, value_test=0.05, nb_steps=10000)

In [10]:
DQN = DQNAgent(model=model, nb_actions=num_actions, memory=memory,
nb_steps_warmup=500, target_model_update=5e-2, policy=policy, gamma=0.99, batch_size=32)

In [11]:
DQN.compile(Adam(lr=1e-3), metrics=['mse'])
DQN.fit(env, nb_steps=100000, visualize=True, verbose=2, callbacks=None)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 100000 steps ...


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


    15/100000: episode: 1, duration: 1.968s, episode steps:  15, steps per second:   8, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.467 [0.000, 1.000],  loss: --, mse: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


...

 99476/100000: episode: 525, duration: 10.050s, episode steps: 500, steps per second:  50, episode reward: 500.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.502 [0.000, 1.000],  loss: 20.964822, mse: 14706.456313, mean_q: 169.888492, mean_eps: 0.100000


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


 99976/100000: episode: 526, duration: 10.053s, episode steps: 500, steps per second:  50, episode reward: 500.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  loss: 20.669573, mse: 13912.456277, mean_q: 165.555759, mean_eps: 0.100000


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


done, took 2014.795 seconds


In [13]:
# saving model weights
DQN.save_weights('dqn_cartpole_weights.h5f', overwrite=True)



In [14]:
DQN.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 388.000, steps: 388
Episode 2: reward: 383.000, steps: 383
Episode 3: reward: 395.000, steps: 395
Episode 4: reward: 387.000, steps: 387
Episode 5: reward: 388.000, steps: 388
